# Additional Baggage Analysis

## Overview

## Business Understanding

## Data Understanding

## Data Preparation

To complete the data preparations we will follow these steps:
- ccc
- ffff
- ccc


### Data Cleaning

In [1]:
# Import necessary libraries to complete EDA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_curve, auc, precision_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn import tree
import helper

%matplotlib inline

pd.options.mode.copy_on_write = True

# Suppress harmless warning for use_inf_as_na
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [2]:
# Load the dataset into the notebook
df = pd.read_csv('data/customer_booking.csv', encoding='latin-1')

In [3]:
# Preview the dataset
df.head()

,num_passengers,sales_channel,trip_type,purchase_lead,length_of_stay,flight_hour,flight_day,route,booking_origin,wants_extra_baggage,wants_preferred_seat,wants_in_flight_meals,flight_duration,booking_complete
0,2,Internet,RoundTrip,262,19,7,Sat,AKLDEL,New Zealand,1,0,0,5.52,0
1,1,Internet,RoundTrip,112,20,3,Sat,AKLDEL,New Zealand,0,0,0,5.52,0
2,2,Internet,RoundTrip,243,22,17,Wed,AKLDEL,India,1,1,0,5.52,0
3,1,Internet,RoundTrip,96,31,4,Sat,AKLDEL,New Zealand,0,0,1,5.52,0
4,2,Internet,RoundTrip,68,22,15,Wed,AKLDEL,India,1,0,1,5.52,0


In [4]:
# View the shape, data type and null counts for each column in df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   num_passengers         50000 non-null  int64  
 1   sales_channel          50000 non-null  object 
 2   trip_type              50000 non-null  object 
 3   purchase_lead          50000 non-null  int64  
 4   length_of_stay         50000 non-null  int64  
 5   flight_hour            50000 non-null  int64  
 6   flight_day             50000 non-null  object 
 7   route                  50000 non-null  object 
 8   booking_origin         50000 non-null  object 
 9   wants_extra_baggage    50000 non-null  int64  
 10  wants_preferred_seat   50000 non-null  int64  
 11  wants_in_flight_meals  50000 non-null  int64  
 12  flight_duration        50000 non-null  float64
 13  booking_complete       50000 non-null  int64  
dtypes: float64(1), int64(8), object(5)
memory usage: 5.3+ 

In [5]:
# Check for duplicates in the train data
df.duplicated().sum()

719

In [6]:
# Removing duplicate rows (keeping the first occurrence)
df = df.drop_duplicates()

Based on the above information, our dataset doesn't have any null values.

Further looking into the dataset we can see that our categorical and quantitative variables split as follow:

- **Categorical variables**: `sales_channel`, `trip_type`, `flight_day`, `route`, `booking_origin`

- **Numerical variables**: `num_passengers`, `purchase_lead`, `length_of_stay`, `flight_hour`, `wants_preferred_seat`, `wants_in_flight_meals`, `flight_duration`, `booking_complete`

The goaal of this notebook is to investigate the relationship between the above features and the target, `wants_extra_baggage`. This will allow to build a model to predict if the customer will choose to add an extra baggage to their flight or not.

#### Data Cleaning: "numerical variables"

In [7]:
# Generate descriptive statistics of numerical variables in the data
df.describe()

,num_passengers,purchase_lead,length_of_stay,flight_hour,wants_extra_baggage,wants_preferred_seat,wants_in_flight_meals,flight_duration,booking_complete
count,49281.000000,49281.000000,49281.000000,49281.000000,49281.000000,49281.000000,49281.000000,49281.000000,49281.000000
mean,1.590187,84.723281,23.053976,9.070676,0.668229,0.295631,0.426635,7.279974,0.149977
std,1.016538,90.410229,33.832149,5.413099,0.470854,0.456331,0.494593,1.496390,0.357052
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.670000,0.000000
25%,1.000000,21.000000,5.000000,5.000000,0.000000,0.000000,0.000000,5.620000,0.000000
50%,1.000000,51.000000,17.000000,9.000000,1.000000,0.000000,0.000000,7.570000,0.000000
75%,2.000000,115.000000,28.000000,13.000000,1.000000,1.000000,1.000000,8.830000,0.000000
max,9.000000,867.000000,778.000000,23.000000,1.000000,1.000000,1.000000,9.500000,1.000000


In [14]:
# Checking how many values we have in each categorical variable
categorical_cols = ['sales_channel', 'trip_type', 'flight_day', 'route', 'booking_origin']
for col in categorical_cols:
    helper.value_count_column(df, col)

sales_channel
Internet    43917
Mobile       5364
Name: count, dtype: int64


trip_type
RoundTrip     48779
OneWay          386
CircleTrip      116
Name: count, dtype: int64


flight_day
Mon    7988
Wed    7562
Tue    7558
Thu    7323
Fri    6685
Sun    6442
Sat    5723
Name: count, dtype: int64


route
AKLKUL    2620
PENTPE     912
MELSGN     833
ICNSIN     793
DMKKIX     729
          ... 
AKLHGH       1
JEDPDG       1
JEDPEN       1
DACPEK       1
MRUXIY       1
Name: count, Length: 799, dtype: int64


booking_origin
Australia               17691
Malaysia                 7055
South Korea              4502
Japan                    3819
China                    3284
                        ...  
Panama                      1
Tonga                       1
Tanzania                    1
Bulgaria                    1
Svalbard & Jan Mayen        1
Name: count, Length: 104, dtype: int64




In [15]:
# Testing all the values in 'booking origin'
test = df['booking_origin'].value_counts(ascending=False)

In [16]:
test.head(50)

booking_origin
Australia               17691
Malaysia                 7055
South Korea              4502
Japan                    3819
China                    3284
Indonesia                2317
Taiwan                   2042
Thailand                 1993
India                    1258
New Zealand              1060
Singapore                1034
United States             453
Vietnam                   386
Macau                     301
Hong Kong                 294
Philippines               265
United Kingdom            173
Brunei                    161
Cambodia                  131
(not set)                  78
Sri Lanka                  73
Italy                      58
France                     58
Canada                     57
Germany                    54
Myanmar (Burma)            51
Netherlands                45
Mauritius                  43
United Arab Emirates       42
Nepal                      39
Bangladesh                 36
Saudi Arabia               35
Russia                   

In [17]:
# Testing all the values in 'route'
test2 = df['route'].value_counts(ascending=False)

In [13]:
test2.head(100)

route
AKLKUL    2620
PENTPE     912
MELSGN     833
ICNSIN     793
DMKKIX     729
          ... 
CMBOOL     126
MYYPER     125
DELPER     123
CGKTPE     122
MNLPER     121
Name: count, Length: 100, dtype: int64